In [1]:
import pandas as pd
import pymongo
import sshtunnel
import os
import numpy as np

In [2]:
server_ip = open('server.ip', 'r').readline().strip()
database = 'Windows-DESKTOP-3HVBJEA'
collection_name = 'HARVEST_DATA'


In [3]:
def upload_csv_to_database(records):
    with sshtunnel.open_tunnel((server_ip, 2222),
                               ssh_username='andrew',
                               ssh_password='a',
                               remote_bind_address=('127.0.0.1', 27017),
                               local_bind_address=('127.0.0.1', 37017)
                               ) as tunnel:
        # Construct MongoDB URI with the local bind address port
        mongo_uri = f"mongodb://localhost:{37017}"

        # connect to mongo uri
        client = pymongo.MongoClient(mongo_uri)
        # get the database
        db = client[database]
        # get the collection
        collection = db[collection_name]
        # insert the data
        collection.insert_many(records)
        # Close the connection
        client.close()


In [4]:
# get all files in the directory
directory = 'fruitSM2'
files = os.listdir(directory)
files.sort()
files

['24.06.24.csv',
 '24.07.01.csv',
 '24.07.10.csv',
 '24.07.15.csv',
 '24.07.22.csv',
 '24.07.29.csv',
 '24.08.05.csv',
 '24.08.12.csv',
 '24.08.19.csv',
 '24.08.26.csv',
 '24.08.30.csv',
 '24.09.02.csv',
 '24.09.06.csv',
 '24.09.09.csv',
 '24.09.13.csv',
 '24.09.19.csv',
 '24.09.23.csv',
 '24.09.27.csv',
 '24.10.04.csv',
 '24.10.07.csv',
 '24.10.11.csv',
 '24.10.14.csv',
 '24.10.18.csv',
 '24.10.28.csv',
 '24.11.01.csv',
 '24.11.04.csv',
 '24.11.08.csv',
 '24.11.11.csv',
 '24.11.15.csv',
 '24.11.19.csv',
 '24.11.22.csv',
 '24.11.25.csv',
 '24.11.29.csv',
 '24.12.02.csv',
 '24.12.06.csv']

In [26]:
# harvest_format = { 
#     "Date": "", # Date format aka file name extracted
#     "Product Name": "", # '죽향저장', '죽향실온', '죽향(실온)', '메리퀸', '죽향(저장)'
#     "Section": "", 
#     "Plant": "",
#     "Fruit vertical length": "",
#     "Fruit horizontal length": "",
#     "Fruit weight": "",
#     "Hardness": "",
#     "Sweetness": "",
#     "Acidity": "",
#     "Color L": "",
#     "Color a": "",
#     "Color b": "",
#     "Remarks": "",
# }   

In [5]:
# # list all name of products name
unique_products = []
for file in files:
    df = pd.read_csv(f'{directory}/{file}', encoding='utf-8')
    products = df['Product Name'].unique()
    for product in products:
        if product not in unique_products:
            print(file)
            print(product)
            unique_products.append(product)

unique_products = list(set(unique_products))
unique_products

24.06.24.csv
메리퀸
24.06.24.csv
죽향실온
24.07.01.csv
죽향저장


['죽향저장', '죽향실온', '메리퀸']

In [6]:
# dump all data into the database
for file in files:  # file format: 21.07.01.csv
    df = pd.read_csv(f'{directory}/{file}', encoding='utf-8')
    datetime = file[:-4]
    # convert to datetime format to store in the database
    datetime = f'20{datetime[:2]}-{datetime[3:5]}-{datetime[6:]}'
    # create new dataframe with new column date from df
    df['Date'] = datetime
    # move Date column to the first column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    # convert dataframe to dictionary
    records = df.to_dict(orient='records')
    # upload to database
    upload_csv_to_database(records)
    print(f'{file} uploaded to database')


24.06.24.csv uploaded to database
24.07.01.csv uploaded to database
24.07.10.csv uploaded to database
24.07.15.csv uploaded to database
24.07.22.csv uploaded to database
24.07.29.csv uploaded to database
24.08.05.csv uploaded to database
24.08.12.csv uploaded to database


2024-12-27 15:31:01,902| ERROR   | Could not establish connection from local ('127.0.0.1', 37017) to remote ('127.0.0.1', 27017) side of the tunnel: 


24.08.19.csv uploaded to database
24.08.26.csv uploaded to database
24.08.30.csv uploaded to database
24.09.02.csv uploaded to database
24.09.06.csv uploaded to database
24.09.09.csv uploaded to database
24.09.13.csv uploaded to database
24.09.19.csv uploaded to database
24.09.23.csv uploaded to database
24.09.27.csv uploaded to database
24.10.04.csv uploaded to database
24.10.07.csv uploaded to database
24.10.11.csv uploaded to database
24.10.14.csv uploaded to database
24.10.18.csv uploaded to database
24.10.28.csv uploaded to database
24.11.01.csv uploaded to database
24.11.04.csv uploaded to database
24.11.08.csv uploaded to database
24.11.11.csv uploaded to database
24.11.15.csv uploaded to database
24.11.19.csv uploaded to database
24.11.22.csv uploaded to database
24.11.25.csv uploaded to database
24.11.29.csv uploaded to database
24.12.02.csv uploaded to database
24.12.06.csv uploaded to database
